## Interactive Computing with Dynamic Compute - ImageStacks

In [ ]:
import descarteslabs.dynamic_compute as dc

In the previous notebook we worked with the `Mosaic` class. Here we will work with the `ImageStack` (insert less awkward segue here)/

In this notebook we will explore the `ImageStack` and its associated functionality to analyze and filter aggregated stacks of imagery. 

_Note_: For those coming from the `Workflows` API, the `ImageStack` is analogous to an `ImageCollection` with the temporal dimension functionality (can be less awkward?).

First we'll define another `map` and `Mosaic` to reference over the Austin, Texas metro area:

In [ ]:
m = dc.map
m.center = 30.2743226, -97.7387934
m.zoom = 13

In [ ]:
s2_mosaic = dc.Mosaic.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "red green blue",
    start_datetime="2023-01-01",
    end_datetime="2023-04-01",
)
_ = s2_mosaic.pick_bands("red green blue").visualize(
    "S2 TCC", m, scales=[[0, 1], [0, 1], [0, 1]]
)

In [ ]:
m

### ImageStacks
The first thing we notice in our `map` viewport is that it's full of clouds! Let's address that by first creating an `ImageStack` with the very same input `from_product_bands` arguments as before:

In [ ]:
s2_stack = dc.ImageStack.from_product_bands(
    "esa:sentinel-2:l2a:v1",
    "red green blue nir swir1",
    start_datetime="2023-01-01",
    end_datetime="2023-04-01",
)
type(s2_stack)

### Filtering ImageStacks
One limitation of the `Mosaic` class is that the _temporal component of our analysis is not exposed_. Now we can _filter_ based off certain metadata properties, such as `cloud_fraction`:

In [ ]:
s2_stack_cloudfree = s2_stack.filter(lambda x: x.cloud_fraction < 0.2)
type(s2_stack_cloudfree)

### Visualizing ImageStacks

If we tried calling `.visualize` on this `ImageStack` now we would not return any imagery, that is because we need to _aggregate_ our data to reduce othat temporal dimension. In the next cell we will call `.median` on our filtered `ImageStack`, but note we have several other operators you can use (e.g. `min`, `mean`, `max`, etc.)

In [ ]:
_ = (
    s2_stack_cloudfree.median(axis="images")
    .pick_bands("red green blue")
    .visualize("S2 TCC Cloudfree", m, scales=[[0, 1], [0, 1], [0, 1]])
)

In [ ]:
m

### Band Ratios with ImageStacks

As we've seen, the enhanced flexibility afforded when working with `ImageStacks` comes with the added complexity of needing to reduce that temporal dimension. 

To wrap this notebook up we will calculate the median [Normalized Difference in Built-up Index](https://d-nb.info/1195147821/34) of our cloud-free study area.

$NDBI = (SWIR1 - NIR) / (SWIR1 + NIR)$

In [ ]:
nir, swir = s2_stack_cloudfree.unpack_bands("nir swir1")
ndbi = (swir - nir) / (swir + nir)
_ = ndbi.median(axis="images").visualize("NDBI", m, scales=[[0, 1], [0, 1], [0, 1]])